In [1]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import sqlalchemy as sal
import psycopg2
import arcgis
from arcgis.gis import GIS
import sys
sys.path.append('../')
from gis_resources import san_diego_county_zips
import os

In [2]:
from arcgis.features import GeoAccessor

In [3]:

#gis = GIS(username='izaslavsky_ucsd')  # this will ask for password. 
print("Enter the Client ID/APP ID generated from Arc GIS:")
app_id = input()
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=app_id)
arcgis.__version__
# Elm5V3upnnV17Q3r

Enter the Client ID/APP ID generated from Arc GIS:


 Elm5V3upnnV17Q3r


Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=PVplAlUR3RIJfX3Q7qZUn35UHTy9qy&allow_verification=false


Enter code obtained on signing in using SAML:  ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'2.1.0'

In [4]:
# df = pd.DataFrame(
#     {'City': ['Buenos Aires', 'Brasilia', 'Santiago', 'Bogota', 'Caracas'],
#      'Country': ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Venezuela'],
#      'Latitude': [-34.58, -15.78, -33.45, 4.60, 10.48],
#      'Longitude': [-58.66, -47.91, -70.66, -74.08, -66.86]})

In [5]:
# gdf = geopandas.GeoDataFrame(
#     df, geometry=geopandas.points_from_xy(df.Longitude, df.Latitude))

In [6]:
# print(gdf.head())

In [7]:
# world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))

# # We restrict to South America.
# ax = world[world.continent == 'South America'].plot(
#     color='white', edgecolor='black')

# # We can now plot our ``GeoDataFrame``.
# gdf.plot(ax=ax, color='red')

# plt.show()

In [8]:
# df

In [ ]:
print("Nourish User:")
nourish_user = input()
print("Nourish Password:")
nourish_pswd = input()

In [10]:
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [11]:
# create a cursor
cur = conn.cursor()
        

# execute a statement
cur.execute('select * from ca_business')

        # display the PostgreSQL database server version
ca_business_result = cur.fetchall()
       
	# close the communication with the PostgreSQL
cur.close()

In [12]:
ca_business_result_list = [(row[1],row[4],row[5],row[7],row[8],row[9]) for row in ca_business_result]
#lat_long_list[:10]

In [13]:
# Create a DataFrame
dataFrame = pd.DataFrame(ca_business_result_list,
              columns=("business_name","latitude", "longitude","zip", "categories","city")
              );

In [14]:
dataFrame

,business_name,latitude,longitude,zip,categories,city
0,Flyers,34.1203512,-116.43687129999999,92284.0,[Gas station],Yucca Valley
1,Del Taco,35.3679968,-119.04427109999999,93309.0,"[Fast food restaurant, American restaurant, Ta...",Bakersfield
2,Lanza's Restaurant,39.2385632,-120.0348415,96143.0,[Italian restaurant],Kings Beach
3,Taqueria La Esquinita,36.3159245,-119.708827,93202.0,[Mexican restaurant],Armona
4,BikeLink eLockers: Ardenwood Park and Ride,37.551193,-122.054447,94555.0,"[Parking lot for bicycles, Park & ride]",Fremont
...,...,...,...,...,...,...
515957,Devil's Dozen Donut Shop,32.7255199,-117.169867,92101.0,[Donut shop],San Diego
515958,Hollywood Star liquor,34.101556099999996,-118.3112881,90028.0,[Liquor store],Los Angeles
515959,RE/MAX Premier Realty,33.6721534,-117.7917674,92604.0,"[Real estate agency, Commercial real estate ag...",Irvine
515960,Mavenlink,33.670243899999996,-117.762992,92618.0,[Software company],Irvine


In [15]:
# since some of the zip are not present in ca_business table
dataFrame = dataFrame[~dataFrame['zip'].isna()]

In [16]:

dataFrame = dataFrame.astype({"zip": int})
dataFrame['zip']

0         92284
1         93309
2         96143
3         93202
4         94555
          ...  
515957    92101
515958    90028
515959    92604
515960    92618
515961    90006
Name: zip, Length: 506583, dtype: int64

In [17]:
# Pick the data for only San Diego County
san_diego_county_zips_lst = san_diego_county_zips()
#df[df['col1'].isin(['a', 'c', 'h'])]
san_diego_county_zips_lst[:10]

[91910, 91911, 91913, 91914, 91915, 91916, 91917, 91931, 91932, 91934]

In [18]:
dataFrame = dataFrame[dataFrame["zip"].isin(san_diego_county_zips_lst)]
dataFrame

,business_name,latitude,longitude,zip,categories,city
7,G-Global - Almacén San Diego,32.55052,-116.944516,92154,"[Customs broker, Distribution service, Transpo...",San Diego
17,Knowhere Games & Comics,33.1379422,-117.17847099999999,92078,"[Comic book store, Collectibles store, Game st...",San Marcos
20,Hilton Garden Inn Carlsbad Beach,33.121439699999996,-117.3260992,92011,[Hotel],Carlsbad
32,Rolando's Taco Shop,32.695846599999996,-117.13851799999999,92113,[Mexican restaurant],San Diego
69,A Childs Place Daycare,32.977381799999996,-117.13286339999999,92129,[Day care center],San Diego
...,...,...,...,...,...,...
515947,Swell Coffee Co.,32.785516799999996,-117.2529374,92109,[Restaurant],San Diego
515948,Hernandez Tire Pros,32.595175,-117.063655,91911,"[Tire shop, Used tire shop, Wheel store]",Chula Vista
515949,Rosecrans plaza,32.7525338,-117.20612009999999,92110,"[Shopping mall, Restaurant]",San Diego
515955,Bank of America ATM (Drive-thru),32.797305099999996,-117.2534197,92109,[ATM],San Diego


In [19]:
#!pip install psycopg2-binary

In [20]:
#!pip show psycopg2-binary

In [21]:
gdf = geopandas.GeoDataFrame(
    dataFrame, geometry=geopandas.points_from_xy(dataFrame.longitude, dataFrame.latitude))

In [22]:
ca_business_sgdf = GeoAccessor.from_geodataframe(gdf, inplace=False, column_name='SHAPE')

In [23]:
#ca_business_sgdf.head(5)
ca_business_sgdf['categories'].fillna('',inplace=True)
ca_business_sgdf['categories']=ca_business_sgdf.categories.apply(lambda x: ','.join(x))


In [24]:
ca_business_sgdf=ca_business_sgdf.drop(['latitude','longitude'], axis=1)

In [25]:
ca_business_sgdf['zip'] = ca_business_sgdf['zip'].astype(str)

In [26]:
ca_business_sgdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46516 entries, 7 to 515957
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   business_name  46516 non-null  object  
 1   zip            46516 non-null  object  
 2   categories     46516 non-null  object  
 3   city           46381 non-null  object  
 4   SHAPE          46516 non-null  geometry
dtypes: geometry(1), object(4)
memory usage: 2.1+ MB


In [27]:
%%time
# Convert back from a SEDF into a feature layer, and publishing on AGOL
my_new_featurelayer = ca_business_sgdf.spatial.to_featurelayer(title="San Diego County Businesses", 
                                                         gis=gis, 
                                                         folder='nourish_gis',
                                                         tags=['SanDiegoCountyBusinesses'])

CPU times: user 32.9 s, sys: 1.83 s, total: 34.7 s
Wall time: 1min 11s


In [28]:
pd.set_option('display.max_colwidth', None)


In [29]:
ca_business_sgdf

,business_name,zip,categories,city,SHAPE
7,G-Global - Almacén San Diego,92154,"Customs broker,Distribution service,Transportation service",San Diego,"{""x"": -116.944516, ""y"": 32.55052, ""spatialReference"": {""wkid"": 4326}}"
17,Knowhere Games & Comics,92078,"Comic book store,Collectibles store,Game store,Toy store",San Marcos,"{""x"": -117.17847099999999, ""y"": 33.1379422, ""spatialReference"": {""wkid"": 4326}}"
20,Hilton Garden Inn Carlsbad Beach,92011,Hotel,Carlsbad,"{""x"": -117.3260992, ""y"": 33.121439699999996, ""spatialReference"": {""wkid"": 4326}}"
32,Rolando's Taco Shop,92113,Mexican restaurant,San Diego,"{""x"": -117.13851799999999, ""y"": 32.695846599999996, ""spatialReference"": {""wkid"": 4326}}"
69,A Childs Place Daycare,92129,Day care center,San Diego,"{""x"": -117.13286339999999, ""y"": 32.977381799999996, ""spatialReference"": {""wkid"": 4326}}"
...,...,...,...,...,...
515947,Swell Coffee Co.,92109,Restaurant,San Diego,"{""x"": -117.2529374, ""y"": 32.785516799999996, ""spatialReference"": {""wkid"": 4326}}"
515948,Hernandez Tire Pros,91911,"Tire shop,Used tire shop,Wheel store",Chula Vista,"{""x"": -117.063655, ""y"": 32.595175, ""spatialReference"": {""wkid"": 4326}}"
515949,Rosecrans plaza,92110,"Shopping mall,Restaurant",San Diego,"{""x"": -117.20612009999999, ""y"": 32.7525338, ""spatialReference"": {""wkid"": 4326}}"
515955,Bank of America ATM (Drive-thru),92109,ATM,San Diego,"{""x"": -117.2534197, ""y"": 32.797305099999996, ""spatialReference"": {""wkid"": 4326}}"
